In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,28202
2,Huelva,Confirmados PDIA 14 días,1260
3,Huelva,Tasa PDIA 14 días,"245,53266948574546"
4,Huelva,Confirmados PDIA 7 días,594
5,Huelva,Total Confirmados,28375
6,Huelva,Curados,23677
7,Huelva,Fallecidos,363


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  28202.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7311.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1284 personas en los últimos 7 días 

Un positivo PCR cada 501 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,28202.0,594.0,1260.0,513170.0,115.751116,245.532669,122.0
Condado-Campiña,8803.0,321.0,645.0,156231.0,205.464985,412.850203,66.0
Huelva-Costa,16488.0,203.0,485.0,289548.0,70.109274,167.502452,51.0
Huelva (capital),7311.0,112.0,287.0,143837.0,77.865918,199.531414,30.0
Almonte,1468.0,100.0,220.0,24507.0,408.046681,897.702697,23.0
Moguer,1072.0,45.0,88.0,21867.0,205.789546,402.432890,10.0
Lucena del Puerto,216.0,25.0,32.0,3261.0,766.636001,981.294082,9.0
Punta Umbría,705.0,15.0,21.0,15355.0,97.688049,136.763269,7.0
Isla Cristina,2047.0,18.0,25.0,21393.0,84.139672,116.860655,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres de San Bartolomé,10.0,5.0,5.0,376.0,1329.787234,1329.787234,0.0
Cumbres Mayores,48.0,16.0,23.0,1749.0,914.808462,1315.037164,0.0
Lucena del Puerto,216.0,25.0,32.0,3261.0,766.636001,981.294082,9.0
Almonte,1468.0,100.0,220.0,24507.0,408.046681,897.702697,23.0
Santa Ana la Real,9.0,3.0,4.0,475.0,631.578947,842.105263,0.0
Galaroza,27.0,8.0,11.0,1382.0,578.871201,795.947902,0.0
Rociana del Condado,497.0,22.0,55.0,7939.0,277.112987,692.782466,2.0
Beas,192.0,18.0,24.0,4341.0,414.651002,552.868003,7.0
Cartaya,1253.0,30.0,96.0,20083.0,149.380073,478.016233,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villarrasa,147.0,1.0,7.0,2211.0,45.228403,316.598824,0.0,0.142857
Almonaster la Real,63.0,1.0,6.0,1788.0,55.928412,335.570470,0.0,0.166667
Cartaya,1253.0,30.0,96.0,20083.0,149.380073,478.016233,2.0,0.312500
Palma del Condado (La),1029.0,6.0,17.0,10801.0,55.550412,157.392834,1.0,0.352941
Niebla,192.0,3.0,8.0,4116.0,72.886297,194.363460,0.0,0.375000
Huelva (capital),7311.0,112.0,287.0,143837.0,77.865918,199.531414,30.0,0.390244
Rociana del Condado,497.0,22.0,55.0,7939.0,277.112987,692.782466,2.0,0.400000
Huelva-Costa,16488.0,203.0,485.0,289548.0,70.109274,167.502452,51.0,0.418557
Aracena,506.0,7.0,16.0,8255.0,84.797093,193.821926,0.0,0.437500
